### rapyuta.io SDK publisher subscriber example.
ROS topic communication between device and cloud instance with rapyuta.io
In this example you can use the package created by publisher-subscirber tutorial.

basic-ros-pubsub:

preinstall: https://userdocs.rapyuta.io/build-solutions/sample-walkthroughs/basic-ros-pubsub/preinstalled-runtime/

docker: https://userdocs.rapyuta.io/build-solutions/sample-walkthroughs/basic-ros-pubsub/docker-runtime/

SDK doc: https://docs.rapyuta.io/python-sdk/

#### 1.Auth, Parameter and File psths.
you can get auth from rapyuta.io console

SDK auth token:https://userdocs.rapyuta.io/developer-guide/tooling-automation/python-sdk/sdk-tokens-parameters/

Device onboarding: https://userdocs.rapyuta.io/developer-guide/manage-machines/onboarding/

In [ ]:
AUTH_TOKEN = ''
PROJECT_ID = ''

#params
retry_count=90
interval=10
device_id = 'e20b04fb-8b2b-422d-9f74-37cd4db042cf'
runtime = 'preinstalled'
# runtime = 'docker'

#file paths of package manifests
base_path = '../io_manifests/basic_ros_publisher_subscriber'
if runtime == 'preinstalled':
    pubsub_manifest = 'preinstalled_ros_publisher_subscriber_manifest.json'
elif runtime == 'docker':
    pubsub_manifest = 'docker_publisher_subscriber_manifest.json'

#### 2. import module and create Client Instance

In [ ]:
from rapyuta_io import Client, DeploymentPhaseConstants, DiskType
from rapyuta_io.clients.package import ROSDistro

client = Client(AUTH_TOKEN, PROJECT_ID)

#### 3. Create routed network

In [ ]:
networks = client.get_all_routed_networks()
network = None
network_name = 'route_network'
for net in networks:
    if net.name == network_name and net.get_status().status == "Running":
        network = net
        print('route_network' + " was found and will be used")
        break
        
if not network:
    network = client.create_cloud_routed_network(network_name, ROSDistro.MELODIC, True)
    network.poll_routed_network_till_ready(retry_count=retry_count, sleep_interval=interval)
    print('route_network' + "was created successfully")

routed_networks = [network]

#### 3. Create package

In [ ]:
import os
import json
import time
from pprint import pprint

def create_package(file_path, packages):
    json_data = open(file_path, 'r')
    json_load = json.load(json_data)
    package_id = None
    for package in packages:
        if package['packageName'] == json_load['name'] and  \
           package['packageVersion'] == json_load['packageVersion']:
            package_id = package['packageId']
            break
    if not package_id:
        res = client.create_package(json_load, retry_limit=3)
        print(json_load['name'] + json_load['packageVersion'] + ' was created')
        package_id = res['packageId']
    else :
        print(json_load['name'] + json_load['packageVersion'] + ' exist')
    
    return package_id

def get_plan_id(pakcage_id, index=0):
    pack = client.get_package(pakcage_id)
    return pack['plans'][index]['planId']

def get_ids(base_path, manifest, packages):
    file_path = os.path.join(base_path, manifest)
    package_id = create_package(file_path, packages)
    plan_id = get_plan_id(package_id) 
    return {
        'packageId': package_id,
        'planId': plan_id        
    }

def wait_till_package_ready(package_id, retry_count=15, sleep_interval=6):
    count = 0
    while client.get_package(package_id)['status'] != 'Complete':
        time.sleep(sleep_interval)
        count+=1
        if count > retry_count:
            return False
    return True


packages = client.get_all_packages()
# pprint(packages)

pubsub_ids =    get_ids(base_path, pubsub_manifest ,packages)    

#### 3.Deploy package

In [ ]:
# Device Talker code snippet
if not wait_till_package_ready(pubsub_ids['packageId'], retry_count, interval):
    exit(0)
package = client.get_package(pubsub_ids['packageId'])
device = client.get_device(device_id)
configuration = package.get_provision_configuration(pubsub_ids['planId'])
configuration.add_device('listener', device)
configuration.add_routed_networks(routed_networks)
deployment = package.provision(deployment_name = "talker-listener",
						provision_configuration = configuration)
res = deployment.poll_deployment_till_ready(retry_count=retry_count, sleep_interval=interval)

#### 7.Deprovision all

In [ ]:
deployment.deprovision()